In [48]:
import pandas as pd
import numpy as np
df = pd.read_csv('jantojun2020.csv', dtype=object)
existing = pd.read_csv('airlines-corgis.csv')

In [92]:
df.head(5).iloc[:,0:]

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,MKT_UNIQUE_CARRIER,MKT_CARRIER_FL_NUM,TAIL_NUM,ORIGIN,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2020,1,1,1,3,1/1/2020,WN,5888,N951WN,ONT,...,95.0,122.0,74.0,363,2,8.0,0.0,27.0,0.0,33.0
1,2020,1,1,1,3,1/1/2020,WN,6276,N467WN,ONT,...,90.0,92.0,71.0,363,2,NaN,NaN,NaN,NaN,NaN
2,2020,1,1,1,3,1/1/2020,WN,4598,N7885A,ONT,...,70.0,68.0,57.0,333,2,NaN,NaN,NaN,NaN,NaN
3,2020,1,1,1,3,1/1/2020,WN,4761,N551WN,ONT,...,75.0,75.0,63.0,333,2,NaN,NaN,NaN,NaN,NaN
4,2020,1,1,1,3,1/1/2020,WN,5162,N968WN,ONT,...,80.0,67.0,57.0,333,2,NaN,NaN,NaN,NaN,NaN


In [118]:
existing.head(5)
existing.iloc[0:5, 11:]

#existing['Statistics.Carriers.Names'].str.split(",", expand=True)
#Tells me max number of airlines
#existing['Statistics.Carriers.Names'].str.split(",", expand=True).notna().cumsum(axis=1).max(axis=1).max()

,Statistics.Carriers.Names,Statistics.Carriers.Total,Statistics.Flights.Cancelled,Statistics.Flights.Delayed,Statistics.Flights.Diverted,Statistics.Flights.On Time,Statistics.Flights.Total,Statistics.Minutes Delayed.Carrier,Statistics.Minutes Delayed.Late Aircraft,Statistics.Minutes Delayed.National Aviation System,Statistics.Minutes Delayed.Security,Statistics.Minutes Delayed.Total,Statistics.Minutes Delayed.Weather
0,"American Airlines Inc.,JetBlue Airways,Contine...",11,216,5843,27,23974,30060,61606,68335,118831,518,268764,19474
1,"American Airlines Inc.,Alaska Airlines Inc.,Co...",14,138,1623,3,7875,9639,20319,28189,24400,99,77167,4160
2,"American Airlines Inc.,Continental Air Lines I...",11,29,1245,15,6998,8287,13635,26810,17556,278,64480,6201
3,"American Airlines Inc.,Continental Air Lines I...",11,73,1562,14,7021,8670,14763,23379,23804,127,65865,3792
4,"American Airlines Inc.,Alaska Airlines Inc.,Co...",13,74,1100,18,5321,6513,13775,13712,20999,120,52747,4141


In [6]:
#Start similar dataframe from new data
new = pd.DataFrame()
new['Airport.Code'] = df["ORIGIN"]

#Create dictionary of airport codes : airport names
codes = pd.Series(existing['Airport.Name'].values, existing['Airport.Code'].values).to_dict()

#Add airport names to the new dataframe
L = list(codes.keys()) + list(codes.values())
new['Airport.Name'] = new['Airport.Code'].str.extract('(' + '|'.join(L) + ')', expand=False).replace(codes)

In [7]:
#Add date information
new['Time.Label'] = df['YEAR'] + '/0' + df['MONTH']
new['Time.Month'] = df['MONTH']
months = {'1':'January', '2':'February', '3':'March', '4':'April', '5':'May', '6':'June'}
new['Time.Month Name'] = new['Time.Month'].replace(months)
new['Time.Year'] = df['YEAR']

#Create dataframe that will hold info about each individual flight
new_indiv = new.copy(deep=True)

#Ignore flights from airports not in original dataset
new = new.dropna(subset=['Airport.Name'])

#Reduce rows to one for each Airport.Code/Time.Label combo (like exisiting dataframe)
new = new.drop_duplicates(subset=['Airport.Code', 'Time.Label'])

In [66]:
#Copy over data from jantojun2020... Currently in delay per minutes
new_indiv['Statictics.# of Delays.Carrier'] = df['CARRIER_DELAY']
new_indiv['Statictics.# of Delays.Late Aircraft'] = df['LATE_AIRCRAFT_DELAY']
new_indiv['Statictics.# of Delays.National Aviation System'] = df['NAS_DELAY']
new_indiv['Statictics.# of Delays.Security'] = df['SECURITY_DELAY']
new_indiv['Statictics.# of Delays.Weather'] = df['WEATHER_DELAY']

#Replace all delays of 0.0 minutes (no delays) with NaN
new_indiv = new_indiv.replace({'0.0': np.nan})

In [144]:
#Create dict to map airline codes to names
airlines = {'AA':'American Airlines Inc.', 'AS':'Alaska Airlines Inc.', 'B6':'JetBlue Airways', 
            'DL':'Delta Air Lines Inc.', 'F9':'Frontier Airlines Inc.', 'G4': np.nan, 'HA':'Hawaiian Airlines Inc.',
            'NK':'Spirit Air Lines', 'UA':'United Air Lines Inc.', 'WN': 'Southwest Airlines Co.'}

#Transfer over airline names
new_indiv['Statistics.Carriers.Names'] = df['MKT_UNIQUE_CARRIER']

#Replace airline codes with names. Allegiant airlines not in original dataset, so its name is repalced with NaN
new_indiv['Statistics.Carriers.Names'] = new_indiv['Statistics.Carriers.Names'].replace(airlines)
pd.set_option('display.max_rows', None)

#FOR LATER: Finds number of airlines for each unique airport/date entry
new_indiv.groupby(['Airport.Code', 'Time.Label'])['Statistics.Carriers.Names'].nunique()

Airport.Code  Time.Label
ABE           2020/01       3
              2020/02       3
              2020/03       3
              2020/04       3
              2020/05       3
              2020/06       3
ABI           2020/01       1
              2020/02       1
              2020/03       1
              2020/04       1
              2020/05       1
              2020/06       1
ABQ           2020/01       7
              2020/02       7
              2020/03       7
              2020/04       7
              2020/05       7
              2020/06       7
ABR           2020/01       1
              2020/02       1
              2020/03       1
              2020/04       1
              2020/05       1
              2020/06       1
ABY           2020/01       1
              2020/02       1
              2020/03       1
              2020/04       1
              2020/05       1
              2020/06       1
ACK           2020/06       3
ACT           2020/01       1
              2

In [7]:
test = new.dropna(subset=['Airport.Name'])
test.reset_index(inplace=True)
test = test.drop('index', axis=1)
test['Time.Month'], test['Time.Year'] = [df['MONTH'], df['YEAR']]
test

,Airport.Code,Airport.Name,Time.Label,Time.Month,Time.Month Name,Time.Year
0,PDX,"Portland, OR: Portland International",2020/01,1,January,2020
1,PHL,"Philadelphia, PA: Philadelphia International",2020/01,1,January,2020
2,PHX,"Phoenix, AZ: Phoenix Sky Harbor International",2020/01,1,January,2020
3,SAN,"San Diego, CA: San Diego International",2020/01,1,January,2020
4,SEA,"Seattle, WA: Seattle/Tacoma International",2020/01,1,January,2020
...,...,...,...,...,...,...
169,BOS,"Boston, MA: Logan International",2020/06,1,June,2020
170,FLL,"Fort Lauderdale, FL: Fort Lauderdale-Hollywood...",2020/06,1,June,2020
171,JFK,"New York, NY: John F. Kennedy International",2020/06,1,June,2020
172,IAD,"Washington, DC: Washington Dulles International",2020/06,1,June,2020
